# Trade Network Analysis
`04oil_panel.ipynb`

Requires:
1. `centrality_scores_DOT.csv` from `02oil_centrality_DOT.ipynb`
2. `imf_data_oil.csv` from `03oil_IFS.ipynb`

Runs an OLS regression on the panel data above, with time fixed effects. The estimated equation is:

$$ \Delta \text{ln} X_{i,t} = \delta_{t} + \beta_1 \Delta \text{ln} X_{i,t-1} + \beta_2 \Delta q_{i,t-1} + \beta_3 ( \Delta p_{t} \times c_{i,t-1}) + \varepsilon_{i,t} $$ 



---

METODO:
1. Improve description
2. Review for errors!
3. Statsmodels instead of pandas!

#### Libraries

In [1]:
import pandas as pd                                         # pandas dataframes used for convenience
from pandas.stats.plm import PanelOLS
import numpy as np
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

In [2]:
ifs = pd.read_csv('imf_data_oil.csv', index_col='date')
ifs1 = ifs.rename(columns={'level_0':'iso2',"('mp', u'@OBS_VALUE')":'mp',"('x', u'@OBS_VALUE')":'x',
                    "('xp', u'@OBS_VALUE')":'xp',"('q', u'@OBS_VALUE')":'q','@OBS_VALUE':'p'
                   })[['iso2','full_name','p','mp','q','x','xp']].reset_index().set_index(['date','iso2']).dropna()
cent = pd.read_csv('centrality_scores_DOT.csv', index_col=['date','iso2'])
panel = ifs1.join(cent).dropna()
panel = panel.reset_index()
panel['date'] = pd.to_datetime(panel['date'],infer_datetime_format=True)
panel = panel.set_index(['date','iso2'])

#### Deflate USD values with price indices

In [3]:
deflators = {'x': 'xp', 'idw': 'mp', 'odw': 'xp'}
for k, v in deflators.iteritems():
    panel[k+'_def'] = np.log(panel[k] * (panel[v] / 100))

panel['q_ch'] = panel['q'].groupby(panel.index.get_level_values(1)).pct_change()

In [4]:
panel['x_ch'] = panel['x_def'].groupby(panel.index.get_level_values(1)).diff(periods=1)
panel['p'] = np.log(panel['p'])
panel['p_ch'] = panel['p'].groupby(panel.index.get_level_values(1)).diff(periods=1)

# Drop unbalanced countries
obs_size = pd.DataFrame(panel.groupby('full_name').size())
panel = panel.join(obs_size, on='full_name')
panel = panel[panel[0] == panel[0].max()]
panel['c'] = ( panel['ecx'] * panel['p_ch'])

panel.to_csv('panel_oil.csv')

In [5]:
#reg  = PanelOLS(y=panel['x_ch'],x=panel[['x_def', 'q_ch', 'c']],time_effects=True)
#reg